In [11]:
import pandas as pd
from datetime import datetime

def get_direct_link(shared_link):
    return 'https://drive.google.com/uc?export=download&id=' + shared_link.split('/')[-2]

link = "https://drive.google.com/file/d/1FAmOiioTsOwuseMKLALDvGDKyVlhpocv/view?usp=drive_link"
direct_link = get_direct_link(link)
wiki_data = pd.read_csv(direct_link, encoding='UTF-8')

link = "https://drive.google.com/file/d/1sZvenOQBsUwKqoinO_FBmUcvCbammn7n/view?usp=drive_link"
direct_link = get_direct_link(link)
odds_data = pd.read_csv(direct_link)

wiki_data['Date'] = pd.to_datetime(wiki_data['Date'])
wiki_data['date'] = wiki_data['Date']
wiki_data['pid'] = wiki_data.groupby('wiki_title').ngroup()

duplicates = wiki_data.duplicated(subset=['pid', 'Date'])
duplicates = duplicates[duplicates].index
wiki_data = wiki_data.drop(duplicates)

wiki_data = wiki_data.set_index(['pid', 'Date']).sort_index()


In [12]:
wiki_data.sort_values(by=['pid', 'date'], inplace=True)

wiki_data['wiki_med365'] = wiki_data.groupby('pid')['wiki'].rolling(window=365, min_periods=0).median().reset_index(0, drop=True)
wiki_data['wiki_yesterday'] = wiki_data.groupby('pid')['wiki'].shift()

wiki_data = wiki_data.dropna(subset=['month', 'year', 'day'])

In [13]:
odds_data['date'] = pd.to_datetime(odds_data['Date'], format='%Y-%m-%d')
odds_data['year'] = odds_data['date'].dt.year
odds_data['month'] = odds_data['date'].dt.month
odds_data['day'] = odds_data['date'].dt.day

wiki_data['date'] = pd.to_datetime(wiki_data['date'], format='%d/%m/%Y')
wiki_data['year'] = wiki_data['date'].dt.year
wiki_data['month'] = wiki_data['date'].dt.month
wiki_data['day'] = wiki_data['date'].dt.day 

In [14]:
wiki_columns = ['date','wiki']+(list(wiki_data.columns[7:20]))

In [15]:
merged_data = pd.DataFrame()
j = -1
names = ''
for i, row in wiki_data.iterrows():
    names = row['original_names']
    winner_mask = odds_data['Winner'].apply(lambda x: x in names)
    temp_data = odds_data[winner_mask].copy()
    temp_data['original_names'] = names
    temp_data = temp_data.merge(pd.DataFrame([row[wiki_columns]]), on=['date'], how='inner')
    merged_data = pd.concat([merged_data,temp_data], ignore_index=True)
    if (i[0] % 50 ==0) & (i[0] > j):
        print(i[0])
        j = i[0]
        

for column in merged_data.columns:
    if column.startswith('wiki'):
        merged_data = merged_data.rename(columns={column: column + '_w'})

0
50
100
150
200
250
300
350
400
450
500
550
600


In [16]:
j = -1
names = ''
for i, row in wiki_data.iterrows():
    names = row['original_names']
    loser_mask = odds_data['Loser'].apply(lambda x: x in names)
    temp_data = odds_data[loser_mask].copy()
    temp_data['original_names'] = names
    temp_data = temp_data.merge(pd.DataFrame([row[wiki_columns]]), on=['date'], how='inner')
    merged_data = pd.concat([merged_data,temp_data], ignore_index=True)
    if (i[0] % 50 ==0) & (i[0] > j):
        print(i[0])
        j = i[0]
    
for column in merged_data.columns:
    if (column.startswith('wiki')) and not column.endswith('_w'):
        merged_data = merged_data.rename(columns={column: column + '_l'})

0
50
100
150
200
250
300
350
400
450
500
550
600


In [17]:
merged_data_l = merged_data[merged_data['wiki_w'].isnull()].reset_index()
merged_data_w = merged_data[merged_data['wiki_l'].isnull()]

combined_df = merged_data_w.combine_first(merged_data_l)

In [18]:
combined_df.to_csv('mens_tennis_merged.csv', index=False)